In [1]:
import sqlite3
con = sqlite3.connect('ClashCric.db')

In [2]:
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE stocks
               (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

In [4]:
import sqlite3
con = sqlite3.connect('example.db')
cur = con.cursor()
for row in cur.execute('SELECT * FROM stocks ORDER BY price'):
    print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [160]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [181]:
def player_records(player_name, team, url, img_src):
    url = f"https://www.espncricinfo.com{url}/"
    print(url)
    requests.get(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html5lib')
    table_data = soup.find('div', class_='more-content overflow-hidden')
    bat_headers = []
    bow_headers = []
    headers = []
    
    for i in table_data.find_all('th'):
        title = i.text
        headers.append(title)
        if '10w' in headers:
            bow_headers = headers
            headers = []
        if  'St' in headers:
            bat_headers = headers
            headers = []

    bat_df = pd.DataFrame(columns = bat_headers)
    bow_df = pd.DataFrame(columns = bow_headers)

    for j in table_data.find_all('tr')[1:]:
            row_data = j.find_all('td')
            row = [tr.text for tr in row_data]
            if len(row) == len(bat_headers):
                bat_df.loc[len(bat_df)] = row
            if len(row) == len(bow_headers):
                bow_df.loc[len(bow_df)] = row
    bat_df['player_name'] = player_name
    bat_df['team'] = team
    bow_df['player_name'] = player_name
    bow_df['team'] = team
    return bat_df, bow_df

In [182]:
def players_list(url):
    url = 'https://www.espncricinfo.com/player/team/australia-2'
    requests.get(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html5lib')
    table_data = soup.find('div', class_='player-index-grid w-100')
    team = []
    for i in table_data.find_all('a'):
        tex = i.select("img[alt]")
        for k in tex:
            img_src = k.get('src')
        h_ref = i.get('href')
        name = "".join(list(filter(lambda x : not x.isnumeric(), str(h_ref).replace('-',' ').replace('/player/',''))))
        if [name.strip(), h_ref, img_src] not in team:
            team.append([name.strip(), h_ref, img_src])
    return team

In [206]:
bat = []
bowl = []
for i in aus[:2]:
    bt, bl = player_records(i[0], 'aus', i[1],i[2])
    bat.append(bt)
    bowl.append(bl)

https://www.espncricinfo.com/player/sean-abbott-398666/
https://www.espncricinfo.com/player/ashton-agar-505120/


In [207]:
df_bat = pd.concat(bat)
df_bowl = pd.concat(bowl)

In [208]:
df_bat

,Format,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100s,50s,4s,6s,Ct,St,player_name,team
0,ODI,2,2,0,7,4,3.50,16,43.75,0,0,0,0,0,0,sean abbott,aus
1,T20I,7,2,1,17,12*,17.00,16,106.25,0,0,1,1,4,0,sean abbott,aus
2,FC,64,91,9,1893,102*,23.08,3529,53.64,1,10,197,42,38,0,sean abbott,aus
3,List A,67,46,6,671,50,16.77,672,99.85,0,1,45,31,29,0,sean abbott,aus
4,T20,94,55,12,447,39,10.39,411,108.75,0,0,26,17,54,0,sean abbott,aus
0,Test,4,7,1,195,98,32.50,351,55.55,0,1,24,3,0,0,ashton agar,aus
1,ODI,15,13,3,236,46,23.60,273,86.44,0,0,25,1,7,0,ashton agar,aus
2,T20I,39,23,4,214,29,11.26,206,103.88,0,0,15,6,26,0,ashton agar,aus
3,FC,61,90,12,2166,114*,27.76,4111,52.68,3,11,264,20,22,0,ashton agar,aus
4,List A,59,46,10,837,64,23.25,927,90.29,0,1,67,24,27,0,ashton agar,aus


In [209]:
df_bowl

,Format,Mat,Inns,Balls,Runs,Wkts,BBI,BBM,Ave,Econ,SR,4w,5w,10w,player_name,team
0,ODI,2,2,90,109,2,1/25,1/25,54.50,7.26,45.00,0,0,0,sean abbott,aus
1,T20I,7,7,108,150,4,2/14,2/14,37.50,8.33,27.00,0,0,0,sean abbott,aus
2,FC,64,115,10722,5594,169,7/45,8/67,33.10,3.13,63.40,7,4,0,sean abbott,aus
3,List A,67,64,3012,2649,108,5/43,5/43,24.52,5.27,27.80,3,2,0,sean abbott,aus
4,T20,94,93,1762,2508,115,5/16,5/16,21.80,8.54,15.30,2,1,0,sean abbott,aus
0,Test,4,8,874,410,9,3/46,5/101,45.55,2.81,97.10,0,0,0,ashton agar,aus
1,ODI,15,15,774,694,14,2/31,2/31,49.57,5.37,55.20,0,0,0,ashton agar,aus
2,T20I,39,39,810,911,42,6/30,6/30,21.69,6.74,19.20,0,2,0,ashton agar,aus
3,FC,61,107,12438,6358,154,6/110,10/141,41.28,3.06,80.70,3,6,2,ashton agar,aus
4,List A,59,58,2905,2488,69,5/39,5/39,36.05,5.13,42.10,0,1,0,ashton agar,aus
